# Description

In [1]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import math
from statsmodels.stats.weightstats import DescrStatsW
from scipy.stats import t
import scipy.optimize as optim
import random

In [2]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 150)

In [3]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/notebooks/calculate_arrival_date'

In [4]:
path = '/Users/alvastrand/Documents/OU/Research/data/'
os.chdir(path)

In [5]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/data'

In [6]:
start_date = '0101'
end_date = '0731'
end_year = '2019'
month = 'Apr'
year_ebird = '2020'
countries_states = 'US_states_east_Mississippi'

In [7]:
subdir = 'output/'
filename = 'obligate_aerial_insectivores_ebird_species_codes.csv'

df_species_codes = pd.read_csv(subdir + filename)

print('len(df_species_codes) =', len(df_species_codes))

len(df_species_codes) = 19


In [8]:
df_species_codes.head(2)

,scientific_name,common_name,species_code,category,taxon_order,order,family,report_as
0,Antrostomus arizonae,Mexican Whip-poor-will,souwpw1,species,3533,Caprimulgiformes,Caprimulgidae,NaN
1,Antrostomus carolinensis,Chuck-will's-widow,chwwid,species,3510,Caprimulgiformes,Caprimulgidae,NaN


In [9]:
for i in range(len(df_species_codes)):
      
    print(i, df_species_codes['species_code'].iloc[i], df_species_codes['common_name'].iloc[i])

0 souwpw1 Mexican Whip-poor-will
1 chwwid Chuck-will's-widow
2 bucnig Buff-collared Nightjar
3 whip-p1 Eastern Whip-poor-will
4 lesnig Lesser Nighthawk
5 comnig Common Nighthawk
6 compoo Common Poorwill
7 whtswi White-throated Swift
8 chiswi Chimney Swift
9 vauswi Vaux's Swift
10 blkswi Black Swift
11 barswa Barn Swallow
12 cavswa Cave Swallow
13 cliswa Cliff Swallow
14 purmar Purple Martin
15 banswa Bank Swallow
16 nrwswa Northern Rough-winged Swallow
17 treswa Tree Swallow
18 vigswa Violet-green Swallow


In [10]:
def get_first_of_season_arrival_day(species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, 
                                    df, list_grid_cells, list_years, *args):
    
    list_grid_cells_first_of_season = []
    list_years_first_of_season = []
    list_t_mad = []
    list_mad = []

    for i in range(len(list_grid_cells)):

#         print('i =', i)

        grid_cell = list_grid_cells[i]
        year = list_years[i]
#         print('(grid_cell, year) =', (grid_cell, year))
        
        # Get data for a given grid cell and year
        df_block_year = df[(df['grid_cell'] == grid_cell) & (df['year'] == year)]
        df_block_year = df_block_year.reset_index(drop=True)
#         print('len(df_block_year) =', len(df_block_year))
        
        start_index = df_block_year[df_block_year['prop'].notna()].head(1).index[0]
#         print('start_index =', start_index)

        df_block_year = df_block_year.iloc[start_index:]
#         print('len(df_block_year) =', len(df_block_year))

        window_size = 7
        # window_size?
        end_index = start_index + window_size - 1
#         print('end_index =', end_index)

        confidence = 0.95

        means = []
        sems = []
        hs = []
        lower_bounds = []
        upper_bounds = []
        
        moving_index = start_index
#         print('moving_index =', moving_index)

        while moving_index < len(df_block_year) - window_size + 1:
            
            df_sample = df_block_year.iloc[range(moving_index, end_index + 1)]
            df_sample = df_sample.dropna()

            if len(df_sample) > 1:

                weights = df_sample['nb_checklists']
                
                dof = 0
                # ?
                # dof = len(df_sample) - 1?
                # Multiply by the total number of checklists?

                weighted_stats = DescrStatsW(df_sample['prop_arcsine'], weights=weights, ddof=dof)

                mean = weighted_stats.mean
                
                sem = weighted_stats.std_mean

                h = sem*t.ppf((1 + confidence)/2, len(df_sample) - 1)
                # ? len(df_sample) - 1?
                lower_bound = mean - h
                upper_bound = mean + h

                means.append(mean)
                sems.append(sem)
                hs.append(h)
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)

            else:

                means.append(np.nan)
                # ?
                sems.append(np.nan)
                # ?
                hs.append(np.nan)
                lower_bounds.append(np.nan)
                upper_bounds.append(np.nan)

            moving_index += 1
            end_index += 1

        # Index of the first mean that is greater than zero
        first_greater_than_zero_index = next((i for i, mean in enumerate(means) if mean > 0), None)
#         print('first_greater_than_zero_index:', first_greater_than_zero_index)
    
#         If there's at least one mean greater than zero:
        if first_greater_than_zero_index != None:
        
#             means[first_greater_than_zero_index] should be greater than zero 
#             (the first value that is greater than zero).
#             print('means[first_greater_than_zero_index]:', means[first_greater_than_zero_index])

            first_greater_than_zero_upper_bound = upper_bounds[first_greater_than_zero_index]
#             print('first_greater_than_zero_upper_bound:', first_greater_than_zero_upper_bound)

            first_greater_than_upper_bound_index = next((i for i, mean in enumerate(means) if 
                                                         mean > first_greater_than_zero_upper_bound), None)
#             print('first_greater_than_upper_bound_index:', first_greater_than_upper_bound_index)

#             If there's at least one proportion that exceeds the upper bound of the confidence interval:
            if first_greater_than_upper_bound_index != None:
        
#                 print('means[first_greater_than_upper_bound_index]:', means[first_greater_than_upper_bound_index])

                index = first_greater_than_upper_bound_index + window_size - 1
#                 print('index:', index)

                t_mad = df_block_year.iloc[index]['julian_day']
#                 print('t_mad:', t_mad)

                mad = df_block_year.iloc[index]['observation_date']
#                 print('mad:', mad)
                
#                 print(df_block_year.iloc[index])
                
#                 print(df_block_year.iloc[range(first_greater_than_zero_index, 
#                                                first_greater_than_zero_index + window_size)])
                
#                 print(df_block_year.iloc[range(first_greater_than_upper_bound_index, 
#                                                first_greater_than_upper_bound_index + window_size)])
                
            # If there aren't any proportions that exceed the upper bound of the confidence interval:
            elif first_greater_than_upper_bound_index == None:

                t_mad = np.nan
                mad = np.nan
        
#             print(len(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt']))
#             print(len(means))
#             print(len(upper_bounds))
            
            assert(len(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
                'observation_date_dt']) == len(means))
        
#             # Means and upper bounds
#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt'], means)
#             plt.scatter(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt'], upper_bounds)
#             plt.show()

#             # Proportions
#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], df_block_year['prop'])
#             plt.show()

#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], df_block_year['prop_arcsine'])
#             plt.show()

        # If there aren't any means greater than zero:
        elif first_greater_than_zero_index == None:

            t_mad = np.nan
            mad = np.nan

        list_grid_cells_first_of_season.append(grid_cell)
        list_years_first_of_season.append(year)
        list_t_mad.append(t_mad)
        list_mad.append(mad)

    df_first_of_season = pd.DataFrame({'grid_cell': list_grid_cells_first_of_season, 
                                       'year': list_years_first_of_season, 
                                       'first_of_season_arrival_day': list_t_mad, 
                                       'first_of_season_arrival_date': list_mad})

    print('len(df_first_of_season):', len(df_first_of_season))
    
    # Filter

    df_first_of_season = df_first_of_season.dropna()

    print('len(df_first_of_season):', len(df_first_of_season))

    subdir = 'eBird/ebd_output/'

    if args != ():

        countries_states = args[0]
        
        if sampled == 1:
            
            string = 'sampled'
            
            random_state = args[1]
            
            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_' + actual_start_date + '_' + actual_end_date + \
            '_complete_zerofilled_grid_cells_proportions_first_of_season_' + str(start_year) + '_' + \
            str(end_year) + '_' + string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'
            
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cells_proportions_first_of_season_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'

        elif sampled == 0:
            
            string = 'not_sampled'
            
            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_' + actual_start_date + '_' + actual_end_date + \
            '_complete_zerofilled_grid_cells_proportions_first_of_season_' + str(start_year) + '_' + \
            str(end_year) + '_' + string + '_rel' + month + '-' + year_ebird + '.csv'
            
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cells_proportions_first_of_season_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_rel' + month + '-' + year_ebird + '.csv'
            
        print('filename =', filename)

    df_first_of_season.to_csv(subdir + filename, index=False)

    return df_block_year, df_first_of_season

In [11]:
def logistic_function(t, a, b, c):
    return c/(1 + a * np.exp(-b * t))

In [12]:
def get_mean_arrival_day(species, start_date, end_date, month, year_ebird, df, list_grid_cells, list_years, *args):

    bounds = (0, [1000000, 100, 1])

    list_grid_cells_logistic = []
    list_years_logistic = []
    list_p0 = []
    list_popt = []
    list_r_squared = []
    list_m_a_day = []
    list_m_a_date = []
    list_lower_bound_c = []
    list_upper_bound_c = []
    list_lower_bound_day = []
    list_upper_bound_day = []
    list_ci_nb_days = []
    cnt_errors = 0

    for i in range(len(list_grid_cells)):
        
#         print('i =', i)
        
        grid_cell = list_grid_cells[i]
        year = list_years[i]
        print('(grid_cell, year) =', (grid_cell, year))

        # Get data for a given grid cell and year
        df_block_year = df[(df['grid_cell'] == grid_cell) & (df['year'] == year)]
        df_block_year = df_block_year.reset_index(drop=True)
#         print('len(df_block_year) =', len(df_block_year))
        
        df_block_year_nas = df_block_year.copy()
        df_block_year = df_block_year.dropna()

        p0 = []
        for j in range(3):
            p0.append(random.uniform(0, 1))
#         print(p0)

        x = df_block_year['julian_day']
        y = df_block_year['prop']

#         try:
        # Nonlinear least squares optimization
        popt, pcov = optim.curve_fit(logistic_function, x, y, bounds=bounds, p0=p0)

        a_optim, b_optim, c_optim = popt
#         print(popt)

        residuals = y - logistic_function(x, a_optim, b_optim, c_optim)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        r_squared = 1 - (ss_res/ss_tot)
        print('r_squared =', r_squared)

        # Time step for mean arrival date
        m_a_day = np.log(a_optim)/b_optim
#         print('m_a_day =', m_a_day)

        # Mean arrival date
        m_a_date_round = df_block_year_nas[
            df_block_year_nas['julian_day'] == round(m_a_day)]['observation_date'].values[0]
#         df_block_year_nas?
        print('m_a_date_round =', m_a_date_round)

#         # Mean arrival date, rounded down
#         m_a_date_floor = df_block_year_nas[
#             df_block_year_nas['t'] == math.floor(m_a_day)]['observation_date'].values[0]
#         print(m_a_date_floor)

#         # Mean arrival date, rounded up
#         m_a_date_ceil = df_block_year_nas[
#             df_block_year_nas['t'] == math.ceil(m_a_day)]['observation_date'].values[0]
#         print(m_a_date_ceil)

        lower_bound_c = (2.5/100)*c_optim
#         print('lower_bound_c =', lower_bound_c)

        upper_bound_c = (97.5/100)*c_optim
#         print('upper_bound_c =', upper_bound_c)

        lower_bound_day = -np.log((1/a_optim)*(c_optim/lower_bound_c - 1))/b_optim
#         print('lower_bound_day =', lower_bound_day)

        upper_bound_day = -np.log((1/a_optim)*(c_optim/upper_bound_c - 1))/b_optim
#         print('upper_bound_day =', upper_bound_day)

#         print('upper_bound_day - lower_bound_day =', upper_bound_day - lower_bound_day)

#         lower_bound_date = df_block_year[df_block_year['prop'] >= lower_bound_c]['observation_date_dt'].min()
#         print('lower_bound_date =', lower_bound_date)

#         upper_bound_date = df_block_year[df_block_year['prop'] <= upper_bound_c]['observation_date_dt'].max()
#         print('upper_bound_date =', upper_bound_date)

#         if r_squared >= 0.8:
#             print('(grid_cell, year) =', (grid_cell, year))
#             print('r_squared =', r_squared)
#             print('m_a_date_round =', m_a_date_round)
            
#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], y)
#             plt.plot(df_block_year['observation_date_dt'], logistic_function(x, a_optim, b_optim, c_optim))
#             plt.show()

        plt.figure(figsize=(12.0, 4.0))
        plt.scatter(df_block_year['observation_date_dt'], y)
        plt.plot(df_block_year['observation_date_dt'], logistic_function(x, a_optim, b_optim, c_optim))
        plt.show()

        list_grid_cells_logistic.append(grid_cell)
        list_years_logistic.append(year)
        list_p0.append(p0)
        list_r_squared.append(r_squared)
        list_m_a_day.append(m_a_day)
        list_m_a_date.append(m_a_date_round)
        list_lower_bound_c.append(lower_bound_c)
        list_upper_bound_c.append(upper_bound_c)
        list_lower_bound_day.append(lower_bound_day)
        list_upper_bound_day.append(upper_bound_day)
        list_ci_nb_days.append(upper_bound_day - lower_bound_day)

#         except (RuntimeError, IndexError) as e:
#             cnt_errors += 1
#             assert(
#                 (e.args[0] == 
#                  'Optimal parameters not found: The maximum number of function evaluations is exceeded.') | 
#                 (e.args[0] == 
#                  'index 0 is out of bounds for axis 0 with size 0'))

#     print(cnt_errors)

    df_logistic = pd.DataFrame({'grid_cell': list_grid_cells_logistic, 'year': list_years_logistic, 
                                'mean_arrival_day': list_m_a_day, 'mean_arrival_date': list_m_a_date, 
                                'r_squared': list_r_squared, 'ci_nb_days': list_ci_nb_days})

    print('len(df_logistic) =', len(df_logistic))
    
    subdir = 'eBird/ebd_output/'
    
    if args != ():

        countries_states = args[0]
        
        if sampled == 1:
            
            string = 'sampled'
            
            random_state = args[1]
        
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_mean_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'
            
            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_complete_zerofilled_grid_cells_proportions_mean_' + str(start_year) + '_' + \
            str(end_year) + '_' + string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'
        
        elif sampled == 0:
            
            string = 'not_sampled'
            
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_mean_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_rel' + month + '-' + year_ebird + '.csv'

            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_complete_zerofilled_grid_cells_proportions_mean_' + str(start_year) + '_' + \
            str(end_year) + '_' + string + '_rel' + month + '-' + year_ebird + '.csv'
        
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cells_proportions_mean_rel' + month + '-' + year_ebird + '_v2.csv'
        
        print('filename =', filename)

#     df_logistic.to_csv(subdir + filename, index=False)
    
    return df_block_year, df_logistic

In [13]:
def get_arrival_days(species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, *args):
    
    subdir = 'eBird/ebd_output/'

    if args != ():

        countries_states = args[0]
        
#         grid_cell = 120
#         grid_cell = 128
#         grid_cell = 136
        
#         grid_cell = 117
#         grid_cell = 147
#         grid_cell = 151
        
        if sampled == 1:
            
            string = 'sampled'
            
            random_state = args[1]
            
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'
            
            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + str(end_year) + '_' + \
            string + '_' + str(random_state) + '_rel' + month + '-' + year_ebird + '.csv'

        elif sampled == 0:
            
            string = 'not_sampled'
            
#             filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + str(start_year) + '_' + \
#             str(end_year) + '_' + string + '_rel' + month + '-' + year_ebird + '.csv'
            
            filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
            '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + str(end_year) + '_' + \
            string + '_rel' + month + '-' + year_ebird + '.csv'
            
        print('filename =', filename)

    df = pd.read_csv(subdir + filename)
    print('len(df) =', len(df))
        
    df['observation_date_dt'] = pd.to_datetime(df['observation_date'])
    
    df_prop_greater_than = df[df['prop'] > 0]
    print('len(df_prop_greater_than):', len(df_prop_greater_than))

    # Calculate the number of proportions greater than 0 for each grid cell and year
    df_prop_greater_than_cnt = df_prop_greater_than[['grid_cell', 'year', 'prop']].groupby(
        ['grid_cell', 'year']).count()
    df_prop_greater_than_cnt = df_prop_greater_than_cnt.reset_index()
    df_prop_greater_than_cnt = df_prop_greater_than_cnt.rename(columns={'prop': 'nb_prop'})
    print('len(df_prop_greater_than_cnt):', len(df_prop_greater_than_cnt))

    # Filter
    
#     df_subset = df_prop_greater_than_cnt[df_prop_greater_than_cnt['nb_prop'] >= 10]

    df_grid_cells_years = df[['grid_cell', 'year']].drop_duplicates().sort_values(by=['grid_cell', 'year'])

    list_grid_cells = list(df_grid_cells_years['grid_cell']) # df_subset
    print('len(list_grid_cells) =', len(list_grid_cells))
    print('len(list(set(list_grid_cells))) =', len(list(set(list_grid_cells))))

    list_years = list(df_grid_cells_years['year']) # df_subset

    df_block_year, df_first_of_season = get_first_of_season_arrival_day(
        species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, df, list_grid_cells, 
        list_years, *args)
    
#     df_block_year, df_logistic = get_mean_arrival_day(
#         species, start_date, end_date, month, year_ebird, df, list_grid_cells, list_years, *args)
    
#     return df, df_prop_greater_than_cnt, df_grid_cells_years, df_block_year, df_logistic

    return df, df_prop_greater_than_cnt, df_grid_cells_years, df_block_year, df_first_of_season

In [14]:
species = 'treswa'
# species = 'barswa'
# species = 'chiswi'

first_year = 2002

# start_year = '2002'
start_year = '2003'

# start_year = '2010'
# start_year = '2012'
# start_year = '2009'
    
# start_year = 2015
# start_year = 2014

actual_start_date = '0101'
actual_end_date = '0630'

sampled = 1

random_state = 1

df_sampled, df_prop_greater_than_cnt_sampled, df_grid_cells_years_sampled, df_block_year_sampled, \
df_first_of_season_sampled = get_arrival_days(
    species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states, 
    random_state)

# df_sampled, df_prop_greater_than_cnt_sampled, df_grid_cells_years_sampled, df_block_year_sampled, \
# df_logistic_sampled = get_arrival_days(
#     species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states, random_state)

# for random_state in range(1, 6):
    
#     df_sampled, df_prop_greater_than_cnt_sampled, df_grid_cells_years_sampled, df_block_year_sampled, \
#     df_first_of_season_sampled = get_arrival_days(
#         species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states, 
#         random_state)

# for start_year in range(first_year, end_year + 1):
    
#     start_year = str(start_year)

#     df_sampled, df_prop_greater_than_cnt_sampled, df_grid_cells_years_sampled, df_block_year_sampled, \
#     df_first_of_season_sampled = get_arrival_days(
#         species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states, 
#         random_state)

filename = ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2003_2019_sampled_1_relApr-2020.csv
len(df) = 18462
len(df_prop_greater_than): 5144
len(df_prop_greater_than_cnt): 102
len(list_grid_cells) = 102
len(list(set(list_grid_cells))) = 6
len(df_first_of_season): 102
len(df_first_of_season): 85
filename = ebd_US_states_east_Mississippi_treswa_0101_0731_0101_0630_complete_zerofilled_grid_cells_proportions_first_of_season_2003_2019_sampled_1_relApr-2020.csv


In [15]:
sampled = 0

df_not_sampled, df_prop_greater_than_cnt_not_sampled, df_grid_cells_years_not_sampled, df_block_year_not_sampled, \
df_first_of_season_not_sampled = get_arrival_days(
    species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states)

# df_not_sampled, df_prop_greater_than_cnt_not_sampled, df_grid_cells_years_not_sampled, df_block_year_not_sampled, \
# df_logistic_not_sampled = get_arrival_days(
#     species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states)

# for start_year in range(first_year, end_year + 1):
    
#     start_year = str(start_year)

#     df_not_sampled, df_prop_greater_than_cnt_not_sampled, df_grid_cells_years_not_sampled, \
#     df_block_year_not_sampled, df_first_of_season_not_sampled = get_arrival_days(
#         species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states)

filename = ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2003_2019_not_sampled_relApr-2020.csv
len(df) = 18462
len(df_prop_greater_than): 10184
len(df_prop_greater_than_cnt): 102
len(list_grid_cells) = 102
len(list(set(list_grid_cells))) = 6
len(df_first_of_season): 102
len(df_first_of_season): 101
filename = ebd_US_states_east_Mississippi_treswa_0101_0731_0101_0630_complete_zerofilled_grid_cells_proportions_first_of_season_2003_2019_not_sampled_relApr-2020.csv


In [16]:
df_sampled.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine,observation_date_dt
0,74,2003,1,2003-01-01,1.0,0.0,0.0,0.0,2003-01-01
1,74,2003,2,2003-01-02,NaN,NaN,NaN,NaN,2003-01-02


In [17]:
df_not_sampled.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine,observation_date_dt
0,74,2003,1,2003-01-01,2.0,0.0,0.0,0.0,2003-01-01
1,74,2003,2,2003-01-02,NaN,NaN,NaN,NaN,2003-01-02


In [18]:
df_prop_greater_than_cnt_sampled.head(2)

,grid_cell,year,nb_prop
0,74,2003,4
1,74,2004,4


In [19]:
df_prop_greater_than_cnt_sampled['nb_prop'].value_counts().to_frame().reset_index().sort_values(by='index')

,index,nb_prop
41,2,1
30,3,1
16,4,3
15,5,3
25,7,2
17,8,2
18,9,2
7,11,3
31,33,1
26,36,2


In [20]:
df_prop_greater_than_cnt_not_sampled.head(2)

,grid_cell,year,nb_prop
0,74,2003,11
1,74,2004,9


In [21]:
df_grid_cells_years_sampled.head(2)

,grid_cell,year
0,74,2003
181,74,2004


In [22]:
df_grid_cells_years_not_sampled.head(2)

,grid_cell,year
0,74,2003
181,74,2004


In [23]:
df_block_year_sampled.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine,observation_date_dt
0,136,2019,1,2019-01-01,7.0,0.0,0.0,0.0,2019-01-01
1,136,2019,2,2019-01-02,1.0,0.0,0.0,0.0,2019-01-02


In [24]:
df_block_year_not_sampled.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine,observation_date_dt
0,136,2019,1,2019-01-01,824.0,0.0,0.0,0.0,2019-01-01
1,136,2019,2,2019-01-02,447.0,0.0,0.0,0.0,2019-01-02


In [25]:
df_first_of_season_sampled.head(2)

,grid_cell,year,first_of_season_arrival_day,first_of_season_arrival_date
9,74,2012,139.0,2012-05-18
13,74,2016,133.0,2016-05-12


In [26]:
df_first_of_season_not_sampled.head(2)

,grid_cell,year,first_of_season_arrival_day,first_of_season_arrival_date
1,74,2004,87.0,2004-03-27
2,74,2005,120.0,2005-04-30


In [ ]:
# df_logistic_sampled.head(2)

In [ ]:
# df_logistic_not_sampled.head(2)

In [ ]:
# df['nb_checklists'].value_counts(sort=False).to_frame().reset_index().sort_values(by=['index']).rename(
#     columns={'index': 'nb_checklists', 'nb_checklists': 'cnt_grid_cells_observation_dates'}).head(2)

In [ ]:
# df_cnt_years = df['year'].value_counts(sort=False).to_frame().reset_index()
# df_cnt_years = df_cnt_years.rename(columns={'index': 'year', 'year': 'cnt_grid_cells_dates'})

In [ ]:
# df_cnt_years.head(2)

In [ ]:
# subdir = 'eBird/ebd_output/'

# filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
# '_complete_zerofilled_grid_cells_proportions_cnt_years_rel' + month + '-' + year_ebird + '.csv'
# print(filename)
        
# df_cnt_years.to_csv(subdir + filename, index=False)

In [ ]:
# species_cnt = 0

# for i in range(len(df_species_codes)):
    
#     print(i)
  
#     species = df_species_codes['species_code'].iloc[i]
#     print(species)
    
#     if ((species == 'souwpw1') | (species == 'bucnig') | (species == 'compoo') | (species == 'whtswi') | 
#         (species == "blkswi") | (species == "treswa")): #
#         continue
    
#     df, df_cnt_checklists, df_cnt_checklists_species, df_prop_greater_than, df_prop_greater_than_cnt, \
# df_subset = get_arrival_days(species, start_date, end_date, month, year_ebird, countries_states)
    
#     species_cnt += 1

In [ ]:
# print(species_cnt)